In [1]:
import pandas as pd
import sqlite3

name = 'Alejandra Camino'

conn = sqlite3.connect("normalized.db")
sql_statement = """ 

    WITH country_rank AS (
        SELECT r.Region, ct.Country, ROUND(SUM(p.ProductUnitPrice * o.QuantityOrdered), 2) AS CountryTotal,
        ROW_NUMBER() OVER (PARTITION BY r.Region ORDER BY SUM(p.ProductUnitPrice * o.QuantityOrdered) DESC) CountryRegionalRank
        From OrderDetail o
        JOIN Customer c ON o.CustomerID = c.CustomerID
        JOIN Product p ON o.ProductID = p.ProductID
        JOIN Country ct ON c.CountryID = ct.CountryID
        JOIN Region r ON r.RegionID = ct.RegionID
        GROUP BY Country
        ORDER BY r.Region ASC, CountryTotal DESC
      )
      SELECT Region, Country, CountryTotal, CountryRegionalRank
      FROM country_rank
      WHERE CountryRegionalRank = 1
      ORDER BY Region ASC
     """
df = pd.read_sql_query(sql_statement, conn)
display(df)
conn.close()


,Region,Country,CountryTotal,CountryRegionalRank
0,British Isles,UK,34560755.49,1
1,Central America,Mexico,25146864.21,1
2,Eastern Europe,Poland,5196640.06,1
3,North America,USA,65090437.08,1
4,Northern Europe,Denmark,10611364.69,1
5,Scandinavia,Finland,10361234.75,1
6,South America,Brazil,47467768.74,1
7,Southern Europe,Spain,23968999.47,1
8,Western Europe,Germany,55556847.68,1


In [2]:
conn.close()